In [ ]:
pip install testcontainers[compose]

In [ ]:
from testcontainers.compose import DockerCompose

with DockerCompose(".",
                   compose_file_name=["docker-compose.yml"],
                   pull=True, build=True) as compose:
    compose.start()
    stdout, stderr = compose.get_logs()
    if stderr:
        print("Errors\n:{}".format(stderr))